# Redes neuronales para datos tabulados 
En el siguiente ejemplo, utilizaremos redes neuronales para estimar el precio de venta de propiedades, en base a un conjunto de datos tabulado que contiene características numéricas y categóricas. Como se mencionó en clases, para manejar las variables categóricas utilizaremos embeddings. Es importante que antes de ejecutar el código, estemos utilizando un _Runtime_ de tipo GPU. Para esto, deben seleccionar en el menú de arriba `Runtime -> Change Runtime Type -> GPU` y luego `Save`.

## 1. Importación de librerías
Es una buena práctica incluir todas las librerías necesarias en el código en el primera celda, con el fin de tener siempre clara qué recursos externos se están utilizando. En este caso, importaremos la librería `pandas` para procesar datos de manera sencilla, la librería `numpy` para el manejo de vectores, `sklearn` para el preprocesamiento del set de datos, y la librería `torch`, que es parte de Pytorch, y permite el uso de redes neuronales. Finalmente, importamos la librería `tqdm` para entregar una visualización clara del proceso.

In [ ]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import tqdm

## 2. Lectura y preprocesamiento de datos
En este paso, utilizamos `pandas` para leer los datos del archivo **train.csv**, eligiendo solo un sobconjunto de las variables que contiene (no hay ningún motivo profundo tras esto último, solo hacer el set de datos más manejable). Una vez cargados los datos, eliminamos todos los registros que tengan valores faltantes mediante el comando `dropna()`. Terminado este proceso, mostramos los cinco primeros registros del set de datos.

In [ ]:
data = pd.read_csv("train.csv", usecols=["SalePrice", "MSSubClass", "MSZoning", "LotFrontage", "LotArea",
                                         "Street", "YearBuilt", "LotShape", "1stFlrSF", "2ndFlrSF"]).dropna()
data.head()

Continuando con el preprocesamiento, almacenamos los nombres de las variables numéricas, de salida y categóricas, transformando estas últimas en valores numéricos mediante el comando `fit_transform` de la clase `LabelEncoder()`. Este proceso identifica los valores únicos de la variable y transforma cada uno de estos a un número natural, lo que facilita su uso posterior en las capas de embedding.

In [ ]:
num_features = ["LotFrontage", "LotArea", "1stFlrSF", "2ndFlrSF"]
output = ["SalePrice"]
cat_features = ["MSSubClass", "MSZoning", "Street", "LotShape", "YearBuilt"]
for cat_col in cat_features:
  data[cat_col] = LabelEncoder().fit_transform(data[cat_col])

## 3. Creación del set de datos
Pytorch provee múltiples facilidades para la carga y manipulación de los datos al momento de entrenar modelos. Para hacer uso de esto, es necesario que los datos se encuentren almacenados en una clase que herede de `Dataset`, por lo que creamos la clase `TabularDataset`. Cualquier clase que herede de `Dataset` debe implementar, además del método `__init__`, los métodos `__len__` y `__getitem__`, que son utilizados por el `DataLoader` (más sobre esto último un poco más adelante).

In [ ]:
class TabularDataset(Dataset):
    def __init__(self, X_num, X_cat, Y):
        self.n = X_num.shape[0]
        self.y = Y.astype(np.float32).values.reshape(-1, 1)

        normalized_X_num = (X_num-X_num.mean())/X_num.std()
        self.x_num = normalized_X_num.astype(np.float32).values

        self.x_cat = X_cat.astype(np.int64).values

    def __len__(self):
        return self.n

    def __getitem__(self, idx):
        return [self.x_num[idx], self.x_cat[idx], self.y[idx]]

Como se aprecia, el método `__init__` es el encargado de almacenar los datos internamente, convirtiendo el formato cuando corresponde. En este caso, los datos son transformados desde `DataFrame` de `pandas`, a `array` multidimensionales de `numpy`. Aprovechando esto, estandarizamos las variables numéricas (línea 6).

Finalmente, dividimos los datos en sets de entrenamiento y test utilizando la función `train_test_split` de `sklearn`, y creamos los objetos correspondientes para cada uno.

In [ ]:
training, test = train_test_split(data, test_size=0.2)
training_dataset = TabularDataset(X_num=training[num_features], X_cat=training[cat_features], Y=np.log(training[output]))
test_dataset = TabularDataset(X_num=test[num_features], X_cat=test[cat_features], Y=np.log(test[output]))

## 4. Creación del modelo
En este paso crearemos una red neuronal utilizando Pytorch. Independiente de la arquitectura de red que se utilice, el proceso es siempre el mismo: i) crear una clase que herede de `nn.Module`, que implemente los métodos `__init__` y `__forward__`. El primero es el encargado de crear la arquitectura _per se_, es decir, crear cada una de las capas de la red y fijar sus tamaños. El segundo método es el encargado definir el flujo de los datos a medida que pasan por la red.

In [ ]:
class TabularMLP(nn.Module):
  def __init__(self, num_input_size, cat_input_emb_size, hidden_size):
      super().__init__()

      self.embeddings = nn.ModuleList([nn.Embedding(x, y) for x, y in cat_input_emb_size])
      total_embedding_size = sum([y for x, y in cat_input_emb_size])

      self.fc1 = torch.nn.Linear(num_input_size+total_embedding_size, hidden_size)
      self.fc2 = torch.nn.Linear(hidden_size, hidden_size)

      self.emb_dropout = torch.nn.Dropout(.1)
      self.dropout1 = torch.nn.Dropout(.1)
      self.dropout2 = torch.nn.Dropout(.1)
      
      self.output = torch.nn.Linear(hidden_size, 1)

  def forward(self, x_num, x_cat):
      x = [embedding(x_cat[:, i]) for i, embedding in enumerate(self.embeddings)]
      x = torch.cat(x, 1)
      x = self.emb_dropout(x)

      x = torch.cat([x, x_num], 1)

      x = F.relu(self.fc1(x))
      x = self.dropout1(x)
      x = F.relu(self.fc2(x))
      x = self.dropout2(x)
      y_ = self.output(x)
      return y_

Algo importante a considerar es que el orden en que las estructuras son definidas en la función `__init__` no tiene relación con el orden en que son llamadas en el _forward pass_ de la red (esto se define en la función `forward`). Tomando eso en consideración, podemos notar como la red está formada una capa del tipo `Embedding`, que es estructurada mediante una lista de módulos, ya que cada variables categórica tiene una matriz de _embedding_ distinta (línea 5). A continuación, en la línea 6, calculamos el tamaño total que tendrán las _features_ generadas por los _embeddings_, en base al parámetro de entrada `cat_input_emb_size` que indica el tamaño de cada matriz de _embedding_. Luego de esto, la definición de la red considera dos capas _fully connected_  (llamadas `nn.Linear` en Pytorch), donde la primera es aplicada a todas las _features_ (numéricas y las generadas por los _embeddings_) y la segunda utiliza como input las generadas por la primera (esto se puede inferir por los tamaños). A continuación, se definen capas de _dropout_ luego de las matrices de _embedding_ y de las capas _fully connected_, todas declaradas con una probilidad de apagado de 0,1. Finalmente, se define la capa de salida, que consiste en una capa _fully connected_ con una sola neurona. 

La función `forward` estructura el _forward pass_ de la red, aplicando inicialmente las matrices de _embedding_ a todas las _features_ categóricas (línea 18). A continuación, concatena la lista de tensores resultantes en uno solo (línea 19) y aplica la capa de _dropout_. Es muy interesante notar que es posible (y recomendado) utilizar la misma variable como entrada y salida para las capas (en este caso, el tensor `x`). Una vez generadas las transformaciones de las _features_ categóricas, estas son concatenadas con las numéricas (línea 22). Posteriormente, en la línea 24, se aplica a esta concatenación de _features_ la primera capa densa, seguida de una no linealidad del tipo ReLU. A diferencia de las otras capas, acá no creamos una capa ReLU, sino que solamente aplicamos la función `F.relu`. Si bien es posible crear capas ReLU en Pytorch, muchas veces no se hace, ya que esta función no tiene parámetros aprendibles y su comportamiento es igual tanto en entrenamiento como en inferencia (a diferencia del _dropout_). Luego de esta aplicación, se continua el flujo esperado, con una capa de _dropout_, luego la segunda densa con ReLU, luego otro _dropout_ y finalmente la capa de salida, que genera el tensor `y_`, que es utilizado como retorno.

## 5. Instanciación del modelo y del manejador de datos
Una vez definida la red, debemos instanciarla para poder entrenarla. Esto se realiza en la siguiente celda:

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

cat_dims = [int(data[col].nunique()) for col in cat_features]
cat_input_emb_size = [(x, min(50, (x + 1) // 2)) for x in cat_dims]

model = TabularMLP(num_input_size=4, cat_input_emb_size=cat_input_emb_size, hidden_size=256).to(device)

La línea 6 de la celda muestra como se instancia la red, utilizando los parámetros derivados del set de datos, como la cantidad de _features_ numéricas y categóricas. Además se define el tamaño de cada matriz de embedding (línea 4). También se deine el tamaño de las capas ocultas, que en este caso es 256 (por ningún motivo en particular). Un aspecto fundamental que siempre es necesario considerar es definir en qué _hardware_ se va a realizar el entrenamiento. Esto no es automático, por lo que primero debemos verificar que exista una GPU disponible y seleccionarla (línea 1, `cuda` es el driver de la GPU para usarla como dispositivo de cómputo). Luego, al declarar el modelo (línea 6) es necesario cargarlo en la GPU, para lo que utilizamos la función `to()`. Dado que anteriormente definimos que la variable `device` contiene la especificación del `hardware` que utilizaríamos, al entregarla como parámetro en la función `to()`, estamos cargando el modelo en la GPU que Colab nos entrega.

Una vez instanciada la red, debemos instanciar el cargador de datos, que se encargará de generar los batches para el entrenamiento de manera transparente. Para lograr esto, creamos un objeto del tipo `DataLoader`, que recibe como parámetro al set de datos de entrenamiento (el para el set de test lo generaremos posteriormente).

In [ ]:
dataloader = DataLoader(training_dataset, batch_size=64, shuffle=True, num_workers=1)

## 6. Entrenamiento
El siguiente paso es generalmente el más complicado de entender, ya que requiere definir explícitamente cada uno de los pasos del entrenamiento. Si bien existen algunas librerías que permiten automatizar y simplificar algunas de estas cosas, es muy importante tener claros todos los pasos, por lo que en este ejemplo cubriremos cada uno:

1. Para empezar, las líneas 1 y 2 definen la cantidad de épocas (cantidad de veces que se recorren los datos) y el _learning rate_ del optimizador.
2. A continuación, en las líneas 4 y 5, se define la función de pérdida (error cuadrático medio, ya que estamos haciendo una regresión) y el optimizador (Adam).
3. Las líneas 7, 9, 28, y 29 tienen que ver con la impresión de la información del entrenamiento en pantalla, por lo que no es necesario ver sus detalles.
4. Las líneas 8 y 12 definen las iteraciones que realizaremos: sobre cada época y sobre cada _batch_. Es importante notar que para cada época, los _batches_ son distintos, es decir, el orden en que se procesan los ejemplos cambia en cada época.
5. Entre las líneas 13 y 15 cargamos en la GPU las _features_ numéricas y categoricas, y el valor a predecir. Todas estas son extraídas de cada _batch_, utilizando el formato indicado en el método `__getitem__` de la clase `TabularDataset`, que definimos anteriormente.
6. Las líneas 18 y 19 realizan el _forward pass_ de los datos del _batch_ por la red (todo el _batch_ de manera simultánea), seguido de el cálculo de la pérdida.
7. Luego, en las líneas 22 y 23, se realiza el _backward pass_ o _backpropagation_ para calcular las derivadas. Este proceso es muy similar al realizado en los ejemplo de código de _backpropagation_ disponibles en el Syllabus.
8. Finalmente, en la línea 26 se realiza un paso de descenso en la dirección del gradiente (aunque en realidad esta dirección depende del optimizador utilizado).

In [ ]:
total_epochs = 2000
lr = 0.01

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

t_epochs = tqdm.notebook.tqdm(range(total_epochs), unit="epoch")
for epoch in t_epochs:
  t_epochs.set_description(f"Epoch {epoch+1}")

  total_loss = 0
  for X_num, X_cat, Y in dataloader:       
    X_num = X_num.to(device)
    X_cat = X_cat.to(device)
    Y = Y.to(device)

    # Forward pass
    Y_ = model(X_num, X_cat)
    loss = criterion(Y_, Y)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()

    # Gradient descent
    optimizer.step()

    total_loss += loss.item()*X_num.size(0)
  t_epochs.set_postfix(loss=total_loss/len(training))

## 7. Test
El procedimiento para hacer test es muy similar al de entrenamiento, con la diferencia que solo es necesario hacer un _forward pass_ por cada batch, ya que no es neceario calcular derivadas ni optimizar los parámetros. Es importante notar que en la línea 1, se indica que el modelo se encuentre en este de evaluación, con el fin de mantener sus parámetros inmutables.

In [ ]:
model.eval()

test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True, num_workers=1)

total_loss = 0
for X_num, X_cat, Y in test_dataloader: 
  X_num = X_num.to(device)
  X_cat = X_cat.to(device)
  Y = Y.to(device)
  Y_ = model(X_num, X_cat)
  loss = criterion(Y_, Y)

  total_loss += loss.item()*X_num.size(0)
avg_loss = total_loss/len(test)
print(f"Avg. Loss = {avg_loss:e}")